In [10]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import sklearn
from sklearn.decomposition import RandomizedPCA, PCA
import os
import numpy as np
import cv2

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'


In [22]:
common_path = '../AK/output'

class_list = os.listdir(common_path)
data_x = np.zeros((2122, 49152))
data_y = np.zeros((2122,1))

y = 0 # y-label
i = 0
for class_folder in class_list:
    for img in os.listdir(common_path + '/' + class_folder):
#         print (common_path+class_folder+ '/'+ img)
        img_1 = cv2.imread(common_path + '/' + class_folder+ '/' +img)
        img_2 = img_1.reshape(49152)
        data_x[i] = img_2
        #print (class_folder, y)
        data_y[i] = [int(y)]
        
        i += 1
#         print (img_1.shape, img_2.shape)
    y += 1
    
from sklearn.model_selection import train_test_split

data_x_scaled = data_x

X_train, X_test, y_train, y_test = train_test_split(data_x_scaled, data_y, stratify = data_y, test_size = 0.05, random_state = 42)    
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)


(2015, 49152) (2015, 1)
(107, 49152) (107, 1)


In [23]:
batch_size = 32
num_classes = 12
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
x_train= X_train.reshape(2015, 128,128,3)
y_train =y_train
x_test= X_test.reshape(107, 128,128,3)
y_test =y_test
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (2015, 128, 128, 3)
2015 train samples
107 test samples


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
63/63 [==============================] - 8s 126ms/step - loss: 2.2528 - acc: 0.2611 - val_loss: 1.9485 - val_acc: 0.3178
Epoch 2/100
63/63 [==============================] - 7s 106ms/step - loss: 1.8653 - acc: 0.3602 - val_loss: 1.6542 - val_acc: 0.4299
Epoch 3/100
63/63 [==============================] - 7s 106ms/step - loss: 1.6522 - acc: 0.4217 - val_loss: 1.4626 - val_acc: 0.5234
Epoch 4/100
63/63 [==============================] - 7s 106ms/step - loss: 1.5275 - acc: 0.4710 - val_loss: 1.3471 - val_acc: 0.5514
Epoch 5/100
63/63 [==============================] - 7s 106ms/step - loss: 1.4180 - acc: 0.5003 - val_loss: 1.1802 - val_acc: 0.5888
Epoch 6/100
63/63 [==============================] - 7s 106ms/step - loss: 1.3328 - acc: 0.5385 - val_loss: 1.0389 - val_acc: 0.6822
Epoch 7/100
63/63 [==============================] - 7s 106ms/step - loss: 1.2305 - acc: 0.5612 - val_loss: 0.9003 - val_acc: 0.7196
Epoch 8/100
63/63 [===============

Epoch 62/100
63/63 [==============================] - 7s 107ms/step - loss: 0.5109 - acc: 0.8401 - val_loss: 0.6112 - val_acc: 0.8131
Epoch 63/100
63/63 [==============================] - 7s 106ms/step - loss: 0.5326 - acc: 0.8252 - val_loss: 0.4663 - val_acc: 0.8692
Epoch 64/100
63/63 [==============================] - 7s 106ms/step - loss: 0.5226 - acc: 0.8203 - val_loss: 0.4397 - val_acc: 0.8692
Epoch 65/100
63/63 [==============================] - 7s 106ms/step - loss: 0.5123 - acc: 0.8278 - val_loss: 0.4730 - val_acc: 0.8411
Epoch 66/100
63/63 [==============================] - 7s 106ms/step - loss: 0.5426 - acc: 0.8104 - val_loss: 0.4477 - val_acc: 0.8972
Epoch 67/100
63/63 [==============================] - 7s 107ms/step - loss: 0.4869 - acc: 0.8417 - val_loss: 0.4470 - val_acc: 0.8692
Epoch 68/100
63/63 [==============================] - 7s 107ms/step - loss: 0.4856 - acc: 0.8482 - val_loss: 0.4964 - val_acc: 0.8598
Epoch 69/100
63/63 [==============================] - 7s 106ms

In [ ]:
model.save('c4.h5')